
# ReaxKit Fort7 (& Xmolout) — End‑to‑End Analysis Notebook

It demonstrates how to:

1. Parse `fort.7` and inspect **metadata, summary DataFrame, and atom frames**.
2. Use `fort7_analyzer` utilities to extract **partial charges**, **CNNs**, **summary metrics**, and **regex‑selected atom features**.
3. Build **connectivity** artifacts: edge lists, adjacency tables, and mean bond orders.
4. Construct **bond‑order time series** (tidy and wide), save CSV, and **plot** a specific bond trace.
5. Detect **bond events** with **threshold + hysteresis + smoothing (MA/EMA)** and export results; generate debug overlay plots.
6. Cross‑reference `fort.7` with `xmolout` to compute **coordination status** across frames and export to CSV.

> **Assumptions:** The files `fort.7` and `xmolout` are present in the working directory.  
> Adjust paths if your files are elsewhere.


In [4]:

# ## Imports
from pathlib import Path

# Handlers
from reaxkit.io.fort7_handler import Fort7Handler
from reaxkit.io.xmolout_handler import XmoloutHandler

# Fort7 analyzers
from reaxkit.analysis.fort7_analyzer import (
    features_atom,
    features_summary,
    partial_charges,
    all_atom_cnn,
    summary_metric_vs_iter,
    coordination_status_over_frames,
)

# Connectivity analyzers
from reaxkit.analysis.connectivity_analyzer import (
    connection_list,
    connection_table,
    connection_stats_over_frames,
    bond_timeseries,
    bond_events,
    debug_bond_trace_overlay,
)

# Plotting
from matplotlib import pyplot as plt

# Display helpers (Jupyter)
from IPython.display import display



## 0) Paths and basic checks

Set paths to required inputs and fail fast if missing.  
Change `fort7_path` and `xmolout_path` if your files live elsewhere.


In [5]:

fort7_path = Path("fort.7")
xmolout_path = Path("xmolout")  # required later for coordination analysis



## 1) Parse `fort.7` and inspect

- Initialize `Fort7Handler`
- Get the summary DataFrame (`handler.dataframe()`)
- Get metadata (`handler.metadata()`)
- Inspect first atom frame (if available)


In [6]:

# === Initialize handler and parse file ===
handler = Fort7Handler(str(fort7_path))

# Summary DataFrame (one row per frame/iteration)
df = handler.dataframe()

# Metadata (dict)
meta = handler.metadata()

# === Print results ===
print("\n=== Metadata ===")
for k, v in meta.items():
    print(f"{k:20s}: {v}")

print("\n=== Summary DataFrame (first 5 rows) ===")
display(df.head())

print("\n=== Number of frames parsed ===")
print(len(handler._frames))

# Optional: show first atom DataFrame structure
if handler._frames:
    print("\n=== Example atom DataFrame (frame 0) ===")
    display(handler._frames[0].head())
else:
    print("\nNo atom frames parsed.")



=== Metadata ===
n_frames            : 9801
n_records           : 9801
simulation_name     : Al2N2_w_001_water

=== Summary DataFrame (first 5 rows) ===


,iter,num_of_atoms,num_of_bonds,total_BO,total_LP,total_BO_uncorrected,total_charge
0,0,276,10,769.047638,148.384461,1065.816560,-3.553089e-06
1,25,276,10,770.079899,148.377542,1066.834984,-3.590440e-06
2,50,276,10,771.317373,148.381676,1068.080725,1.244342e-05
3,75,276,10,774.193396,148.380626,1070.954649,-1.002509e-05
4,100,276,10,777.124771,148.374960,1073.874692,2.611040e-08



=== Number of frames parsed ===
9801

=== Example atom DataFrame (frame 0) ===


,atom_num,atom_type_num,atom_cnn1,atom_cnn2,atom_cnn3,atom_cnn4,atom_cnn5,atom_cnn6,atom_cnn7,atom_cnn8,...,BO4,BO5,BO6,BO7,BO8,BO9,BO10,sum_BOs,num_LPs,partial_charge
0,1,3,19,21,28,60,0,0,0,0,...,0.490,0.0,0.0,0.0,0.0,0.0,0.0,2.105,1.001,-1.372
1,2,3,19,25,27,63,0,0,0,0,...,0.508,0.0,0.0,0.0,0.0,0.0,0.0,2.086,1.001,-1.416
2,3,3,17,18,19,52,0,0,0,0,...,0.513,0.0,0.0,0.0,0.0,0.0,0.0,2.070,1.001,-1.401
3,4,3,17,23,25,51,0,0,0,0,...,0.438,0.0,0.0,0.0,0.0,0.0,0.0,2.028,1.002,-1.367
4,5,3,20,25,32,0,0,0,0,0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.799,1.020,-1.358



## 2) Feature extraction via `fort7_analyzer`

This cell reproduces your feature‑extraction flow:

1. **Partial charges** across all frames (`partial_charges`).
2. **CNN columns** sampled every 10th frame (`all_atom_cnn`).
3. **Summary slice** of `iter`, `total_charge`, `total_BO` for iterations `1000..2000`.
4. **Regex‑selected atom features** (all `BO\\d+` columns + `partial_charge`).
5. **Ready‑to‑plot** `total_charge` vs iteration series.


In [7]:

h = handler  # reuse same handler

# 1) All partial charges over all frames
pc = partial_charges(h)
print("Partial charges shape:", pc.shape)
display(pc.head())

# 2) All atom_cnn* columns, but only every 10th frame
cnn = all_atom_cnn(h, frames=range(0, len(h.dataframe()), 10))
print("CNN (sampled) shape:", cnn.shape)
display(cnn.head())

# 3) total_charge and total_BO for iterations 1000..2000
summary_slice = features_summary(h, ["iter", "total_charge", "total_BO"],
                                 iterations=range(1000, 2001))
print("Summary slice shape:", summary_slice.shape)
display(summary_slice.head())

# 4) Arbitrary atom-level features via regex
atom_features = features_atom(h, [r"^BO\\d+$", "partial_charge"], regex=True)
print("Atom features (regex) shape:", atom_features.shape)
display(atom_features.head())

# 5) A ready-to-plot series of total_charge vs iteration
tc = summary_metric_vs_iter(h, "total_charge")
print("total_charge vs iteration (head):")
display(tc.head())


Partial charges shape: (2705076, 4)


,frame_idx,iter,atom_idx,partial_charge
0,0,0,0,-1.372
1,0,0,1,-1.416
2,0,0,2,-1.401
3,0,0,3,-1.367
4,0,0,4,-1.358


CNN (sampled) shape: (270756, 18)


,frame_idx,iter,atom_idx,atom_cnn1,atom_cnn2,atom_cnn3,atom_cnn4,atom_cnn5,atom_cnn6,atom_cnn7,atom_cnn8,atom_cnn9,atom_cnn10,atom_cnn11,atom_cnn12,atom_cnn13,atom_cnn14,atom_cnn15
0,0,0,0,19,21,28,60,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,0,0,1,19,25,27,63,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,0,0,2,17,18,19,52,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,0,0,3,17,23,25,51,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,0,0,4,20,25,32,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


Summary slice shape: (41, 4)


,frame_idx,iter,total_charge,total_BO
0,40,1000,0.000003,882.284366
1,41,1025,0.000010,882.252925
2,42,1050,-0.000003,881.763796
3,43,1075,0.000012,881.947405
4,44,1100,-0.000015,881.067579


Atom features (regex) shape: (2705076, 4)


,frame_idx,iter,atom_idx,partial_charge
0,0,0,0,-1.372
1,0,0,1,-1.416
2,0,0,2,-1.401
3,0,0,3,-1.367
4,0,0,4,-1.358


total_charge vs iteration (head):


,iter,total_charge
0,0,-3.553089e-06
1,25,-3.590440e-06
2,50,1.244342e-05
3,75,-1.002509e-05
4,100,2.611040e-08



## 3) Connectivity artifacts

- Edge lists with bond‑order thresholds (all frames or sampled frames)
- One‑frame **adjacency table** (use with care for large systems)
- **Mean BO** per pair across a sampled set of frames


In [8]:

# 1) Edge list for all frames, keep BO >= 0.3, undirected bonds
elist = connection_list(h, min_bo=0.3, undirected=True)
print("Edge list (all frames) rows:", len(elist))
display(elist.head())

# 2) Edge list for frames 0..999 stepping by 10
elist_10 = connection_list(h, frames=range(0, 1000, 10), min_bo=0.2)
print("Edge list (0..999 step 10) rows:", len(elist_10))
display(elist_10.head())

# 3) One-frame adjacency table (wide), use sparingly
adj0 = connection_table(h, frame=0, min_bo=0.3)
print("Adjacency (frame 0) shape:", adj0.shape)
display(adj0.iloc[:10, :10])  # show a small corner

# 4) Mean BO across frames for each pair (network summary)
mean_edges = connection_stats_over_frames(
    h, frames=range(0, len(h.dataframe()), 5),
    min_bo=0.3, how="mean"
)
print("Mean-edges rows:", len(mean_edges))
display(mean_edges.head())


Edge list (all frames) rows: 5396126


,frame_idx,iter,src,dst,bo,j
0,0,0,1,19,0.558,-1
1,0,0,1,21,0.545,-1
2,0,0,1,28,0.512,-1
3,0,0,1,60,0.490,-1
4,0,0,2,19,0.504,-1


Edge list (0..999 step 10) rows: 57507


,frame_idx,iter,src,dst,bo,j
0,0,0,1,19,0.558,-1
1,0,0,1,21,0.545,-1
2,0,0,1,28,0.512,-1
3,0,0,1,60,0.490,-1
4,0,0,2,19,0.504,-1


Adjacency (frame 0) shape: (198, 192)


dst,17,18,19,20,21,22,23,24,25,26
src,,,,,,,,,,
1,0.000,0.000,0.558,0.000,0.545,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.504,0.000,0.000,0.000,0.000,0.000,0.544,0.000
3,0.536,0.519,0.502,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.557,0.000,0.000,0.000,0.000,0.000,0.450,0.000,0.583,0.000
5,0.000,0.000,0.000,0.578,0.000,0.000,0.000,0.000,0.657,0.000
6,0.000,0.000,0.000,0.519,0.624,0.000,0.000,0.000,0.000,0.000
7,0.000,0.000,0.000,0.595,0.000,0.593,0.572,0.000,0.000,0.000
8,0.000,0.555,0.000,0.000,0.513,0.572,0.000,0.000,0.000,0.000
9,0.000,0.523,0.000,0.000,0.000,0.000,0.000,0.573,0.000,0.000


Mean-edges rows: 1279


,src,dst,value
0,1,19,0.512432
1,1,21,0.562487
2,1,28,0.570895
3,1,60,0.508733
4,2,19,0.495602



## 4) Bond‑order time series & plotting

- Build **tidy** and **wide** time series tables
- Save wide table to CSV
- **Plot** a specific bond's BO vs iteration (single plot)


In [9]:

# 1) Tidy time series for all bonds across all frames (undirected, zero-filled)
ts = bond_timeseries(h)
print("Bond time series (tidy) shape:", ts.shape)
display(ts.head())
# Columns: frame_idx, iteration (or 'iter'), src, dst, bo

# 2) Wide matrix for every 10th frame; set small BO (< 0.0) to 0
ts_wide = bond_timeseries(h, frames=range(0, len(h.dataframe()), 10),
                          bo_threshold=0.0, as_wide=True)
print("Bond time series (wide) shape:", ts_wide.shape)
display(ts_wide.head())
ts_wide.to_csv("bond_timeseries.csv")
print("Saved CSV: bond_timeseries.csv")

# 3) Focus on a subset of iterations (e.g., 1000..2000)
ts_sub = bond_timeseries(h, iterations=range(1000, 2001))
print("Subset time series shape:", ts_sub.shape)

# 4) Plot a specific bond (e.g., 5–62) vs iteration:
b = ts[(ts["src"] == 5) & (ts["dst"] == 62)].sort_values("iter")
print(b["iter"].head(10))
print(b["bo"].head(10))

if not b.empty:
    plt.figure()
    plt.plot(b["iter"], b["bo"])
    plt.xlabel("Iteration")
    plt.ylabel("Bond Order")
    plt.title("Bond 5–62: BO vs Iteration")
    plt.grid(True)
    plt.show()
else:
    print("No entries found for bond 5–62.")


Bond time series (tidy) shape: (13241151, 5)


,frame_idx,iter,src,dst,bo
0,0,0,1,19,0.558
1,0,0,1,21,0.545
2,0,0,1,28,0.512
3,0,0,1,60,0.490
4,0,0,2,19,0.504


Bond time series (wide) shape: (981, 1193)


,,1-19,1-21,1-28,1-60,2-19,2-25,2-27,2-63,2-260,3-17,...,260-265,261-262,262-263,263-264,265-266,267-268,269-270,271-272,273-274,275-276
frame_idx,iter,,,,,,,,,,,,,,,,,,,,,
0,0,0.558,0.545,0.512,0.490,0.504,0.544,0.529,0.508,0.0,0.536,...,0.0,0.899,0.0,0.893,0.905,0.914,0.900,0.905,0.920,0.898
10,250,0.510,0.509,0.571,0.520,0.508,0.530,0.539,0.536,0.0,0.502,...,0.0,0.902,0.0,0.918,0.935,0.912,0.907,0.925,0.904,0.923
20,500,0.480,0.505,0.616,0.485,0.552,0.396,0.546,0.577,0.0,0.558,...,0.0,0.933,0.0,0.933,0.908,0.908,0.925,0.920,0.917,0.923
30,750,0.492,0.511,0.576,0.572,0.576,0.425,0.566,0.501,0.0,0.551,...,0.0,0.883,0.0,0.918,0.891,0.912,0.920,0.923,0.922,0.891
40,1000,0.519,0.491,0.512,0.638,0.593,0.412,0.539,0.478,0.0,0.543,...,0.0,0.896,0.0,0.888,0.917,0.904,0.904,0.916,0.906,0.888


Saved CSV: bond_timeseries.csv
Subset time series shape: (36162, 5)
Series([], Name: iter, dtype: int64)
Series([], Name: bo, dtype: float64)
No entries found for bond 5–62.



## 5) Bond **events** (threshold + hysteresis + smoothing)

- Compute event table for **all bonds** using **MA** smoothing
- Compute event table for a **specific bond** using **EMA**
- Save per‑bond events to CSV
- Generate **debug overlay** plots (EMA vs MA) and save PNGs


In [10]:

src = 5
dst = 62

# All bonds with MA smoothing:
ev = bond_events(h, threshold=0.1, hysteresis=0.05, smooth="ma", window=8, min_run=5)
print("Events (MA) — head:")
display(ev.head())
print(f"Events for bond {src}-{dst} (MA):")
display(ev[(ev["src"] == src) & (ev["dst"] == dst)].head())

# A specific bond with EMA smoothing:
ev_src_dst = bond_events(
    h, src=src, dst=dst,
    threshold=0.1, hysteresis=0.05,
    smooth="ema", window=8, min_run=5
)
ev_src_dst.to_csv(f"ev_{src}_{dst}.csv", index=False)
print(f"Saved per-bond events: ev_{src}_{dst}.csv")
display(ev_src_dst.head())

# Debug overlay plots (saved as PNG)
debug_bond_trace_overlay(
    h, src, dst, smooth="ema", window=8, hysteresis=0.05, threshold=0.10,
    xaxis="iteration", save=f"bond_{src}_{dst}_overlay_ema.png", min_run=5
)
debug_bond_trace_overlay(
    h, src, dst, smooth="ma", window=8, hysteresis=0.05, threshold=0.10,
    xaxis="iteration", save=f"bond_{src}_{dst}_overlay_ma.png", min_run=5
)
print("Saved debug overlays:",
      f"bond_{src}_{dst}_overlay_ema.png, bond_{src}_{dst}_overlay_ma.png")


Events (MA) — head:


,src,dst,event,frame_idx,iter,x_axis,bo_at_event,threshold,hysteresis
0,1,60,breakage,5745,143625,143625,0.038750,0.1,0.05
1,1,60,formation,5960,149000,149000,0.126875,0.1,0.05
2,1,60,breakage,5972,149300,149300,0.037875,0.1,0.05
3,1,60,formation,6020,150500,150500,0.125000,0.1,0.05
4,1,60,breakage,6030,150750,150750,0.038625,0.1,0.05


Events for bond 5-62 (MA):


,src,dst,event,frame_idx,iter,x_axis,bo_at_event,threshold,hysteresis


Saved per-bond events: ev_5_62.csv


,src,dst,event,frame_idx,iter,x_axis,bo_at_event,threshold,hysteresis


No data for bond 5-62.
No data for bond 5-62.
Saved debug overlays: bond_5_62_overlay_ema.png, bond_5_62_overlay_ma.png



## 6) Coordination status over frames (Fort7 ⟷ Xmolout)

- Initialize `XmoloutHandler`
- Compute coordination statistics across frames using expected valences
- Save results to `coordinations.csv`


In [12]:

if not xmolout_path.exists():
    raise FileNotFoundError("xmolout is required for this section. Update `xmolout_path` and rerun.")
    
xh = XmoloutHandler(str(xmolout_path))
f7 = h  # alias for clarity

# Customize per system if needed:
valences = {"Mg": 2, "Zn": 2, "O": 2, "Al": 3, "N": 3, "He": 2, 'H': 1}

df_coord = coordination_status_over_frames(
    f7, xh,
    valences=valences,
    threshold=0.25,
    frames=None,           # or e.g., range(0, 200, 5)
    require_all_valences=True,
)

print("Coordination status (head):")
display(df_coord.head())

df_coord.to_csv("coordinations.csv", index=False)
print("Saved CSV: coordinations.csv")
# Columns: frame_index, iteration, atom_id, atom_type, sum_BOs, valence, delta, status, status_label


Coordination status (head):


,frame_index,iter,atom_id,atom_type,sum_BOs,valence,delta,status,status_label
0,0,0,1,N,2.105,3.0,-0.895,-1,under
1,0,0,2,N,2.086,3.0,-0.914,-1,under
2,0,0,3,N,2.070,3.0,-0.930,-1,under
3,0,0,4,N,2.028,3.0,-0.972,-1,under
4,0,0,5,N,1.799,3.0,-1.201,-1,under


Saved CSV: coordinations.csv
